In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import requests
import re
import subprocess
import glob
from astropy.io import fits
from astropy.wcs import WCS

from astropy.wcs import FITSFixedWarning

import warnings # To ignore our problems
warnings.filterwarnings('ignore', category=FITSFixedWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning) #Shhhhh, ignore our problems

%matplotlib widget

In [2]:

class download_TESS_FFIs:
    def __init__(self, target_directory, sector, aggressive = True):

        if target_directory[-1] != '/':
            target_directory += '/'

        self.target_directory = target_directory

        if isinstance(sector, int):
            self.sectors = [sector]
        else:
            try:
                self.sectors = [int(s) for s in sector]
            except:
                raise ValueError("Please provide a valid sector number.")

        self.flag = False
        self.close = False
        self.aggressive = aggressive

        self.df = pd.DataFrame(columns=['Sector', 'Camera', 'CCD', 'RA_center', 'DEC_center', 'ROTATION',
                                        'RA_corner1', 'DEC_corner1', 'RA_corner2', 'DEC_corner2', 
                                        'RA_corner3', 'DEC_corner3', 'RA_corner4', 'DEC_corner4'])

        self.run_all()

    def run_all(self):

        for sector in self.sectors:
            print(f"Downloading FFIs for sector {sector}...")
            self.sector = sector

            folder_name = f"{self.target_directory}s{str(self.sector).zfill(4)}"

            fold = self._make_folder(folder_name)
            if (fold == True) & (self.aggressive == True):
                print(f"FFIs for sector {sector} already downloaded.")
                continue

            self.folder_name = folder_name

            self.download_sector_scripts()
            if not self.flag:
                self.get_sector_ffis()
                self.download_sector_ffis()

            else:
                print(f"Failed to download the FFIs for sector {sector}.")
                pass

    def download_sector_scripts(self):
        """
        Download the FFIs for a given sector using the curl script provided by MAST.
        
        Parameters
        ----------
        sector : int
            The sector number to download the FFIs for.
        """
        url = f'https://archive.stsci.edu/missions/tess/download_scripts/sector/tesscurl_sector_{self.sector}_ffic.sh'

        response = requests.get(url)

        if response.status_code == 200:
            file_content = response.text
            self.file_content = file_content
        else:
            print(f"Failed to download the file. Status code: {response.status_code}")
            self.flag = True

    def get_sector_ffis(self):
        content = self.file_content.split('\n')

        sector_str = str(self.sector).zfill(4)
        pattern = re.compile(rf'(tess\d+)-s{sector_str}-\d-\d-\d+-s_ffic\.fits')

        unique_tess_ids = set()
        for line in content:
            match = pattern.search(line)
            if match:
                unique_tess_ids.add(match.group(1))

        self.filtered_lines_by_tess = {}
        for tess_id in unique_tess_ids:
            filtered_lines = [line for line in content if tess_id in line]
            self.filtered_lines_by_tess[tess_id] = filtered_lines

    def download_sector_ffis(self):
        for _, lines in self.filtered_lines_by_tess.items():
            self.filtered_lines = lines
            self._download_files()

            self._gather_files()

            if self.gather_wcs(self.file_list[0]):
                print('WCS information found... processing WCS information.')
                self.wcs_process()
                break
            else:
                self._delete_files()

    def wcs_process(self):
        for file in self.file_list:
            self.file = file

            boolean = self.gather_wcs(file)

            coord_list = self.get_wcs()
            self.file = file

            data_split = file.split('-')

            sector = int(data_split[1].split('s')[1])
            camera = int(data_split[2])
            ccd = int(data_split[3])

            sector_data = [sector, camera, ccd]
            coord_list = self.get_wcs()
            
            sector_data.extend(coord_list)
            self.df.loc[len(self.df)] = sector_data

    def gather_wcs(self, file):
        with fits.open(file) as hdul:
            self.wcs = WCS(hdul[1].header)
            self.shape = hdul[1].data.shape
            hdr = hdul[1].header

            if 'CD1_1' in hdr:
                cd = np.array([[hdr['CD1_1'], hdr['CD1_2']],
                            [hdr['CD2_1'], hdr['CD2_2']]])
            elif 'PC1_1' in hdr:
                pc = np.array([[hdr['PC1_1'], hdr['PC1_2']],
                            [hdr['PC2_1'], hdr['PC2_2']]])
                cdelt = np.array([hdr['CDELT1'], hdr['CDELT2']])
                cd = pc * cdelt[:, np.newaxis]
            else:
                return False
            
            self.rotation_angle = np.arctan2(cd[1, 0], cd[0, 0]) * 180 / np.pi

            return True
        
    def get_wcs(self):
        ra_center, dec_center = self.wcs.all_pix2world(self.shape[1]/2, self.shape[0]/2, 1)
        calc_footprint = self.wcs.calc_footprint()

        coord_list = [ra_center, dec_center] +  [self.rotation_angle] + list(calc_footprint.flatten())
        return coord_list

    def _download_files(self):
        for line in self.filtered_lines:
            match = re.search(r'-o (\S+)', line)
            if match:
                filename = match.group(1)
                last_part = line.split('https:')[-1]
                first_part = line.split('https:')[0]
                new_line = first_part.replace(filename, self.folder_name + f"/{filename}") + 'https:' + last_part
                subprocess.run(new_line, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    def _gather_files(self):
        self.file_list = sorted(glob.glob(f"{self.folder_name}/*.fits"))

    def _delete_files(self):
        for filename in os.listdir(self.folder_name):
            file_path = os.path.join(self.folder_name, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)

    def _make_folder(self, folder_name):
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
            return False
        else:
            return True


In [3]:
sector = np.arange(1, 80)
# sector = 9
target_directory = "/Users/zgl12/Python_Scripts/SynDiff/TESS_Images/"

x = download_TESS_FFIs(target_directory, sector, aggressive = False)

WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS information.
WCS information found... processing WCS inform

In [4]:

x.df

,Sector,Camera,CCD,RA_center,DEC_center,ROTATION,RA_corner1,DEC_corner1,RA_corner2,DEC_corner2,RA_corner3,DEC_corner3,RA_corner4,DEC_corner4
0,1,1,1,319.4839874994128,-41.03857318238723,16.178684,313.777217,-47.871234,310.092203,-36.701889,324.468061,-33.330235,330.129431,-44.413344
1,1,1,2,334.4746587131874,-36.43184132336889,26.415267,330.344064,-44.336617,324.664608,-33.277880,337.720584,-28.385630,344.128531,-38.562134
2,1,1,3,328.6997053978832,-25.17258034492085,-158.671958,332.049749,-17.867487,337.598475,-28.164441,324.550516,-33.034760,320.192557,-21.767718
3,1,1,4,315.326393680347,-29.20348439691792,-163.994506,320.013395,-21.820102,324.359209,-33.090412,310.044938,-36.449224,307.641963,-25.139192
4,1,2,1,333.20741801221794,-63.42319030246114,24.364041,324.750778,-70.676065,317.244403,-59.600483,338.371126,-54.343546,352.506681,-64.922769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,79,3,4,253.70612377056437,79.15886774918089,-179.894578,305.020796,82.168925,274.195324,72.047745,233.675261,72.144535,206.505038,82.224912
1260,79,4,1,85.15490387383218,89.24755012908427,153.568387,205.132069,82.457130,118.233155,80.889959,33.683317,80.901775,304.940766,82.082144
1261,79,4,2,348.59449737243176,77.57044660851824,-92.047279,305.649440,81.947453,32.869645,80.799265,5.308080,70.358777,329.929774,71.303045
1262,79,4,3,32.003631576773685,71.94393398922603,42.612310,31.282841,64.016190,6.098719,70.257742,34.102021,80.576122,57.686599,70.057228


In [5]:
df = x.df

df.to_csv('TESS_FFI_Coordinates.csv', index = False)

In [1]:
2458819.314363 - 2400000.5

58818.81436299998